In [1]:
#Import dependencies
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from flask import Flask, render_template, redirect

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\ljean\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [3]:
############ PART 1 - NASA MARS NEWS  ##############################

#Establish Mars News URL to be scraped
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
html = browser.html
response = requests.get(url)

# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html, 'html.parser')

In [4]:
#pull in results to enable easier parsing
results = soup.find_all('div', class_='content_title')
results

[<div class="content_title">Mars Now</div>,
 <div class="content_title"><a href="/news/8888/new-study-challenges-long-held-theory-of-fate-of-mars-water/" target="_self">New Study Challenges Long-Held Theory of Fate of Mars' Water</a></div>,
 <div class="content_title"><a href="/news/8886/nasas-perseverance-mars-rover-mission-honors-navajo-language/" target="_self">NASA's Perseverance Mars Rover Mission Honors Navajo Language</a></div>,
 <div class="content_title"><a href="/news/8885/perseverance-rovers-supercam-science-instrument-delivers-first-results/" target="_self">Perseverance Rover's SuperCam Science Instrument Delivers First Results</a></div>,
 <div class="content_title"><a href="/news/8882/nasas-perseverance-drives-on-mars-terrain-for-first-time/" target="_self">NASA's Perseverance Drives on Mars' Terrain for First Time</a></div>,
 <div class="content_title"><a href="/news/8881/media-telecon-materials-march-5-2021/" target="_self">Media Telecon Materials - March 5, 2021</a></di

In [5]:
# Return the latest news title and paragraph stored as variables
news_title = soup.find_all('div', class_='content_title')[1].text
news_p = soup.find_all('div', class_='article_teaser_body')[0].text
print('----------------------------')
print(news_title)
print('------------------------------------------------------------------------')
print(news_p)
print('----------------------------')

----------------------------
New Study Challenges Long-Held Theory of Fate of Mars' Water
------------------------------------------------------------------------
The new science results indicate that a large quantity of the Red Planet’s water is trapped in its crust rather than having escaped into space.
----------------------------


In [10]:
############ PART 2 - JPL Mars Space Images  ##############################
#Use splinter to navigate the site and find the image url for the current Featured Mars Image 

#establish base url link
jpl_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(jpl_url)
jpm_html = browser.html
response = requests.get(jpl_url)

# Create BeautifulSoup object; parse with 'html.parser'
soup_image = bs(jpm_html, 'html.parser')

In [16]:
#Find the image url to the full size .jpg image.
image = soup_image.find_all('img', class_='headerimage fade-in')[0]['src']
image

'image/featured/mars3.jpg'

In [18]:
#assign the url string to a variable called featured_image_url.
cleaned_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/'
featured_image_url = cleaned_url + image
print(featured_image_url)


https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars3.jpg


In [ ]:
############ PART 3 - Mars Facts  ##############################
#Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
#Use Pandas to convert the data to a HTML table string.

In [19]:
#Access the mars facts url
facts_url = 'https://space-facts.com/mars/'

In [21]:
#load the url in using pandas
tables = pd.read_html(facts_url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [22]:
#Convert table into a dataframe displaying facts about the planet including Diameter, Mass, etc.
facts_df = tables[0]
facts_df.head()

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"


In [25]:
#convert dataframe back to html table string
html_table = facts_df.to_html()
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>0</th>\n      <th>1</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    <

In [27]:
#save a copy of the html
facts_df.to_html('table.html')